## Vector Clock implementation using Python 3
A vector clock is an algorithm for generating a partial ordering of events in a distributed system and detecting causality violations.
In a distributed system consist of N processes each process is going to have a(n) vector(array) of N logical clocks(simply counters), one clock per process; The values of these counters will be updated by the following rules:
* Initialize all logical clocks to zero in each vector.
* Each time a process experiences an internal event, it increments its own logical clock in the vector by one.
* Each time a process sends a message, it increments its own logical clock in the vector by one and then piggyback a copy of its own vector to the message.
* Each time a process receives a message, it increments its own logical clock in the vector by one and updates each element in its vector by taking the maximum of the value in its own vector clock and the value in the vector in the received message.

### Implementation Details:
* Instead of using an array of N counters a dictionary (map) is used in which the dictionary key is the process id. This way we no longer need to define the number of processes a head and we could have variable number of processes.
* Every event takes two argument an arbitrary name (which should be uniqe for that class) and duration, The duration is updating the actual cpu counter of the process
* Send method also is implemented like an event so it has thoes two arguments that event has(name is replaced with message-msg- here plus it also implements the receive operation as well so it takes the receiver process as an extra argument
* The receive method is implied in the send which actually make sens (There should be a send event happening for the receive side to take place)
* Colon ':' character shouldn't be used for process or event name

In [47]:
class process(object):
    
    events=dict() # static dictionary for process class to store all events
    
    @staticmethod
    def vClock

    def __init__(self, name):
        self.id = name
        self.vClocks=dict()
        self.vClocks[self.id]=0 # initialize its logical clock to zero
        self.timer = 0
        
    def event(self, name, duration=30):
        self.vClocks[self.id]+=1
        self.timer += duration
        events[self.id+':'+name] = self.vClocks

    # send and receive events both are happening in this method
    def send(self, msg, receiver, duration=10):
        # send event
        self.vClocks[self.id]+=1
        self.timer += duration
        events[self.id+':'+nmsg] = self.vClocks
        # send happens before receive
        # receive event
        receiver.vClocks[receiver.id]+=1
        for key,val in self.vClocks.items():
            if key in receiver.vClocks:
                receiver.vClocks[key] = max(receiver.vClocks[key], val)
            else:
                receiver.vClocks[key] = val
        receiver.timer += duration
        events[receiver.id+':'+nmsg] = receiver.vClocks
        
    def printVectorClocks(self):
        print("Process:",self.id,"timer:", self.timer)
        for key,val in sorted(self.vClocks.items()):
            print(key,"->",val)
        print("=======================")
    
    def eventCauses(self, name):
        for key, val in events.items():
            if 

SyntaxError: unexpected EOF while parsing (<ipython-input-47-187509e53eaa>, line 41)

### Here is a simple implementation for the example of vector clocks shown in following figure.

In [46]:
A = process(name="A")
B = process(name="B")
C = process(name="C")

C.send("ce", B, 20)
B.send("t2", A, 30)
A.send("kl", B, 14)
B.send("ha", C, 23)
C.send("er", A, 17)
B.send("iu", C, 30)
C.send("n7", A, 21)


A.printVectorClocks()
B.printVectorClocks()
C.printVectorClocks()

Process: A timer: 82
A : 4
B : 5
C : 5
Process: B timer: 117
A : 2
B : 5
C : 1
Process: C timer: 111
A : 2
B : 5
C : 5
